In [3]:
import pandas as pd
import os

In [4]:
os.environ['KAGGLE_USERNAME'] = "XXXXXXXX" # username from the json file
os.environ['KAGGLE_KEY'] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" # key from the json file
!kaggle datasets download -d ellipticco/elliptic-data-set
!unzip elliptic-data-set.zip
!mkdir elliptic_bitcoin_dataset_cont

 94% 137M/146M [00:03<00:00, 39.6MB/s]
100% 146M/146M [00:03<00:00, 46.8MB/s]
Archive:  elliptic-data-set.zip
  inflating: elliptic_bitcoin_dataset/elliptic_txs_classes.csv  
  inflating: elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv  
  inflating: elliptic_bitcoin_dataset/elliptic_txs_features.csv  


In [5]:
df_edge = pd.read_csv('elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv')
df_class = pd.read_csv('elliptic_bitcoin_dataset/elliptic_txs_classes.csv')
df_features = pd.read_csv('elliptic_bitcoin_dataset/elliptic_txs_features.csv',header=None)

Step1

In [6]:
df_features["contiguosId"] = [i for i in range(len(df_features))]
df_original_contigous = df_features[[0,"contiguosId"]]
df_features1 = df_features.drop(columns=[0])
cols = df_features1.columns.to_list()
cols = cols[-1:]+ cols[0:-1]
df_features1 = df_features1[cols] 
df_original_contigous = df_original_contigous.rename(columns={0: "originalId"})

In [7]:
df_features1["contiguosId"] = pd.to_numeric(df_features1["contiguosId"], downcast="float")
df_features1[1] = pd.to_numeric(df_features1[1], downcast="float")

Step2

In [8]:
dict_contigous = {df_original_contigous['originalId'][i]:df_original_contigous['contiguosId'][i] for i in range(len(df_original_contigous))}
df_class["txId"] = [dict_contigous[df_class.txId[i]] for i in range(len(df_class))]
df_class["class"] = [1 if df_class["class"][i] == 1 else -1 if df_class["class"][i] == "unknown" else 0 for i in range(len(df_class))]

In [9]:
df_class['txId'] = pd.to_numeric(df_class['txId'], downcast="float")
df_class['class'] = pd.to_numeric(df_class['class'], downcast="float")

Step3

In [ ]:
df_nodetime = df_features[["contiguosId",1]]
df_nodetime[1] = [df_nodetime[1][i] - 1 for i in range(len(df_nodetime))]
df_nodetime=df_nodetime.rename(columns={"contiguosId":"txId",1: "timestep"})

Step4

In [ ]:
df_edge["txId1"] = [dict_contigous[df_edge.txId1[i]] for i in range(len(df_edge))]
df_edge["txId2"] = [dict_contigous[df_edge.txId2[i]] for i in range(len(df_edge))]
dict_time = {df_nodetime['txId'][i]:df_nodetime['timestep'][i] for i in range(len(df_original_contigous))}
df_edge["timestep"] = [dict_time[i] for i in df_edge["txId1"]]

In [ ]:
df_edge['timestep'] = pd.to_numeric(df_edge['timestep'], downcast="float")
df_edge.head()

Save all dataframes to csv and clone the repository

In [ ]:
!git clone https://github.com/rosieiiiii/EvolveGCN EvolveGCN
!cd EvolveGCN

!mkdir ./EvolveGCN/data/elliptic_temporal
!mkdir ./EvolveGCN/data/elliptic_temporal/elliptic_bitcoin_dataset_cont
!mkdir log

In [ ]:
df_features1.to_csv('./EvolveGCN/data/elliptic_temporal/elliptic_bitcoin_dataset_cont/elliptic_txs_features.csv',index=False,header=None)
df_original_contigous.to_csv('./EvolveGCN/data/elliptic_temporal/elliptic_bitcoin_dataset_cont/elliptic_txs_orig2contiguos.csv',index=False,header=True)
df_class.to_csv('./EvolveGCN/data/elliptic_temporal/elliptic_bitcoin_dataset_cont/elliptic_txs_classes.csv',index=False,header=True)
df_nodetime.to_csv('./EvolveGCN/data/elliptic_temporal/elliptic_bitcoin_dataset_cont/elliptic_txs_nodetime.csv',index=False,header=True)
df_edge.to_csv('./EvolveGCN/data/elliptic_temporal/elliptic_bitcoin_dataset_cont/elliptic_txs_edgelist_timed.csv',index=False,header=True)

In [ ]:
!tar -czvf "./EvolveGCN/data/elliptic_temporal/elliptic_bitcoin_dataset_cont.tar.gz" "./EvolveGCN/data/elliptic_temporal" 

Train EvolveGCN

In [ ]:
!python ./EvolveGCN/run_exp.py --config_file ./EvolveGCN/experiments/parameters_elliptic_egcn_h.yaml